 <span style="font-size:16pt;">Name: Lamine Deen</span>

1.

In [462]:
import numpy as np

#class for ridge lbf regression
class RidgeRegression:
  def __init__(self, lambda_value):
      self.lambda_value = lambda_value
      self.theta = None

  # constructor
  def fit(self, X, y):
      n, d = X.shape
      # Add column of 1 for constant term in matrix
      X_design = np.c_[np.ones(n), X]

      # Formula for finding theta coeff in part (b)
      self.theta = np.linalg.inv(X_design.T @ X_design + self.lambda_value * np.identity(d + 1)) @ X_design.T @ y

  # function to predict after learning regression coefficients
  def predict(self, X):
      n, _ = X.shape
      # Add column of 1 for constant term in matrix
      X_design = np.c_[np.ones(n), X]

      # Make predictions using the learned coefficients
      predictions = X_design @ self.theta
      return predictions


 <span style="font-size:16pt;">2.</span>

In [463]:
import pandas as pd

# Read the CSV file
file_path = 'Mount_Pleasant_Real_Estate_Data.csv'
df = pd.read_csv(file_path)

# Display the number of rows and columns
print("Number of rows and columns before dropping columns:")
print(df.shape)

# # Display the headers
# print("\nHeaders:")
# print(df.columns)

# Drop columns with names "Misc Exterior" and "Amenities"
columns_to_drop = ["Misc Exterior", "Amenities", "ID", 'Baths - Full', 'Baths - Half']
df = df.drop(columns=columns_to_drop)

# Drop rows with missing values
df = df.dropna()

# Display the number of rows and columns after dropping columns
print("\nNumber of rows and columns after dropping columns:")
print(df.shape)
# Display the headers
print("\nHeaders:")
print(df.columns)



# print(df.head)


Number of rows and columns before dropping columns:
(270, 24)

Number of rows and columns after dropping columns:
(245, 19)

Headers:
Index(['List Price', 'Duplex?', 'Bedrooms', 'Baths - Total', 'Stories',
       'Subdivision', 'Square Footage', 'Year Built', 'Acreage', 'New Owned?',
       'House Style', 'Covered Parking Spots', 'Has Pool?', 'Has Dock?',
       'Fenced Yard', 'Screened Porch?', 'Golf Course?', 'Fireplace?',
       'Number of Fireplaces'],
      dtype='object')


 <span style="font-size:13pt;">Encode yes and no answers as binary data type (1 or 0)</span>

In [464]:
# Display columns with 'yes' and 'no' answers
yes_no_columns = df.columns[df.isin(['Yes', 'No']).any()]

# Print columns with 'yes' and 'no' answers
print("Columns with 'yes' and 'no' answers:")
print(yes_no_columns)

# Encode 'yes' as 1 and 'no' as 0
df[yes_no_columns] = df[yes_no_columns].replace({'Yes': 1, 'No': 0})

# Display the DataFrame after encoding
print("\nDataFrame after encoding 'yes' as 1 and 'no' as 0:")
print(df.head(15))

# Display columns with 'yes' and 'no' answers
yes_no_columns = df.columns[df.isin(['Yes', 'No']).any()]

# Print columns with 'yes' and 'no' answers
print("Columns with 'yes' and 'no' answers:")
print(yes_no_columns)

# # Identify columns that do not contain integers
# non_int_columns = df.columns[~df.applymap(lambda x: isinstance(x, int)).all()]

# # Print column names with non-integer values
# print("Columns that do not contain integers:")
# print(non_int_columns)

Columns with 'yes' and 'no' answers:
Index(['Duplex?', 'New Owned?', 'Has Pool?', 'Has Dock?', 'Fenced Yard',
       'Screened Porch?', 'Golf Course?', 'Fireplace?'],
      dtype='object')

DataFrame after encoding 'yes' as 1 and 'no' as 0:
   List Price  Duplex?  Bedrooms  Baths - Total  Stories    Subdivision  \
0    $369,900        1       3.0            2.5      2.0  Carolina Park   
1    $375,000        1       3.0            2.5      2.0  Carolina Park   
2    $769,900        0       4.0            3.5      2.0  Carolina Park   
3    $699,990        0       4.0            3.5      2.0  Carolina Park   
4    $436,625        0       4.0            3.0      2.0  Carolina Park   
5    $446,900        0       3.0            2.5      2.0  Carolina Park   
6    $575,000        0       4.0            3.5      2.0  Carolina Park   
7    $575,000        0       4.0            3.0      2.0  Carolina Park   
8    $464,900        0       3.0            2.5      2.0  Carolina Park   
9    $469

 <span style="font-size:13pt;">Special handling of any column without integer type</span>

In [465]:
# deal with prices column
# Strip non-numeric characters and parse 'List Price' to integer type
df['List Price'] = df['List Price'].replace('[^\d]', '', regex=True).astype(int)

# deal with bath total column by mulx2 to keep information consistent
# Multiply all elements in 'Baths - Total' by 2 and parse to integer type
df['Baths - Total'] = (df['Baths - Total'].astype(float) * 2).astype(int)

#deal with columns that can be parse to int type without any damage to structure of data
# List of columns to parse to int type
columns_easy = ['Bedrooms', 'Number of Fireplaces', 'Year Built', 'Square Footage', 'Stories']

# Parse elements in the specified columns to int type
df[columns_easy] = df[columns_easy].astype(int)

#deal with acreage column to keep info reliability
# Multiply all elements in 'Acreage' by 100 and parse to int type
df['Acreage'] = (df['Acreage'].astype(float) * 100).astype(int)

#deal with parking spots columns to take into account houses that need a certain amount of parking
#spots per bedroom (found at specific indexes)
# Retrieve values in 'Covered Parking Spots' column at indices 50, 51, and 52
parking_values = df.loc[[50, 51, 52], 'Covered Parking Spots']

# Multiply by 3 and round up to the nearest integer
updated_parking_values = np.ceil(parking_values * 3).astype(int)

# Update the original DataFrame with the new values
df.loc[[50, 51, 52], 'Covered Parking Spots'] = updated_parking_values

# Parse elements in the Parking spot column to int type
df['Covered Parking Spots'] = df['Covered Parking Spots'].astype(int)

# deal with columns with string data
# use one hot encoding
print(df.shape)
# Columns to one-hot encode
columns_to_encode = ['Subdivision', 'House Style']

# Use one-hot encoding
df_encoded = pd.get_dummies(df, columns=['Subdivision', 'House Style'], drop_first=True)
print(df_encoded.shape)
print(df.columns)
print(df_encoded.columns)
# Convert boolean values to integers (True -> 1, False -> 0)
df_encoded = df_encoded.astype(int)


print(df.head(10))
print(df_encoded.head(10))


# Remove the original columns from the original DataFrame
df = df_encoded

# # # Rename the one-hot encoded columns
# # df_encoded.columns = [f"{col}_encoded" for col in df_encoded.columns]

# # Concatenate the original DataFrame with the one-hot encoded columns
# df = pd.concat([df, df_encoded], axis=1)
# print(df.shape)






# Identify columns that contain strings
string_columns = df.columns[df.applymap(lambda x: isinstance(x, str)).any()]

# Print column names with string values
print("Columns that contain strings:")
print(string_columns)


# Identify columns that do not have integers
non_int_columns = df.select_dtypes(exclude=['int']).columns

# Print column names with non-integer data type
print("Columns that do not have integers:")
print(non_int_columns)

print(df_encoded.columns)
# print(df["List Price_encoded"])s


(245, 19)
(245, 29)
Index(['List Price', 'Duplex?', 'Bedrooms', 'Baths - Total', 'Stories',
       'Subdivision', 'Square Footage', 'Year Built', 'Acreage', 'New Owned?',
       'House Style', 'Covered Parking Spots', 'Has Pool?', 'Has Dock?',
       'Fenced Yard', 'Screened Porch?', 'Golf Course?', 'Fireplace?',
       'Number of Fireplaces'],
      dtype='object')
Index(['List Price', 'Duplex?', 'Bedrooms', 'Baths - Total', 'Stories',
       'Square Footage', 'Year Built', 'Acreage', 'New Owned?',
       'Covered Parking Spots', 'Has Pool?', 'Has Dock?', 'Fenced Yard',
       'Screened Porch?', 'Golf Course?', 'Fireplace?', 'Number of Fireplaces',
       'Subdivision_Dunes West', 'Subdivision_Park West',
       'House Style_Colonial', 'House Style_Condo Regime',
       'House Style_Condominium', 'House Style_Contemporary',
       'House Style_Cottage', 'House Style_Craftsman', 'House Style_Patio',
       'House Style_Ranch', 'House Style_Townhouse',
       'House Style_Traditional'],

 <span style="font-size:13pt;">Use the next 2 cells to find the weird data that needs special handling (not a integer type)</span>

In [466]:
# # Print all elements of the 'Covered Parking Spots' column
# bedrooms_column = df['Covered Parking Spots']
# print("All elements of the 'Covered Parking Spots' column:")
# print(bedrooms_column)

# # Filter elements with decimal value equal to 0 in the 'Covered Parking Spots' column
# filtered_bedrooms = df.loc[df['Covered Parking Spots'].astype(float) % 1 != 0, 'Covered Parking Spots']

# # Print the filtered elements
# print("Elements of 'Covered Parking Spots' with decimal value equal to 0:")
# print(filtered_bedrooms)

 

In [467]:
#Parse to int
# 'Bedrooms', 'Number of Fireplaces', 'Year Built', 'Square Footage', 'Stories', 

#special filtering
# 'Covered Parking Spots'

# encoding strings
# 'House Style', 'Subdivision'

# floats to ints
# 'Acreage'

 <span style="font-size:13pt;">Splitting data for Learning</span>

In [468]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = df.drop('List Price', axis=1)
y = df['List Price']

trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.4, random_state=0)
valX, testX, valY, testY = train_test_split(testX, testY, test_size=0.5, random_state=0)

# Standardize the features using StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
valX = scaler.transform(valX)

 <span style="font-size:13pt;">Regular linear regression (Ordinary Least Square)</span>

In [469]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Fit a linear regression model on the training set
model = LinearRegression()
model.fit(trainX, trainY)

# Predict house prices on the validation set
y_pred_val = model.predict(valX)

# Evaluate the model on the validation set using Mean Squared Error (MSE)
mse_val = mean_squared_error(valY, y_pred_val)

# Evaluate the model on the validation set using R-squared
r2_val = r2_score(valY, y_pred_val)

# Evaluate the model on the validation set using Mean Absolute Error (MAE)
mae_val = mean_absolute_error(valY, y_pred_val)

# Display the metrics on the validation set
print("Mean Squared Error on the Validation Set:", mse_val)
print("R-squared on the Validation Set:", r2_val)
print("Mean Absolute Error on the Validation Set:", mae_val)

# # Get the coefficients and corresponding feature names
# coefficients = model.coef_
# feature_names = feature_names = X.columns.tolist()

# # Create a DataFrame to display coefficients
# coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# # Display the coefficients along with the corresponding feature names
# print(coefficients_df)

# # Identify the strongest feature based on absolute coefficient value
# strongest_feature_index = np.argmax(np.abs(coefficients))
# strongest_feature = feature_names[strongest_feature_index]
# strongest_feature_coefficient = coefficients[strongest_feature_index]
# print(f"The strongest feature is: {strongest_feature} with coefficient: {strongest_feature_coefficient}")

Mean Squared Error on the Validation Set: 12805842582.21691
R-squared on the Validation Set: 0.8750513048835247
Mean Absolute Error on the Validation Set: 80795.62894775019


 <span style="font-size:13pt;">Ridge Regression with lambda 0.1 : 15 : ++0.2 </span>

In [470]:
from sklearn.linear_model import Ridge

# Define a range of lambda values
lambda_values = [i/10 for i in range(1, 150, 2)]

# Loop over different values of lambda
for lambda_val in lambda_values:
    # Fit a ridge regression model on the training set
    ridge_model = Ridge(alpha=lambda_val)
    ridge_model.fit(trainX, trainY)

    # Predict house prices on the validation set
    y_pred_val = ridge_model.predict(valX)

    # Evaluate the model on the validation set using Mean Squared Error (MSE)
    mse_val = mean_squared_error(valY, y_pred_val)

    # Evaluate the model on the validation set using R-squared
    r2_val = r2_score(valY, y_pred_val)

    # Evaluate the model on the validation set using Mean Absolute Error (MAE)
    mae_val = mean_absolute_error(valY, y_pred_val)

    # Display the metrics on the validation set for the current lambda value
    print(f"\nLambda = {lambda_val}")
    print("Mean Squared Error on the Validation Set:", mse_val)
    print("R-squared on the Validation Set:", r2_val)
    print("Mean Absolute Error on the Validation Set:", mae_val)




# from sklearn.model_selection import GridSearchCV

# # Create Ridge regression model
# ridge_model = Ridge()

# # Define a range of lambda values
# param_grid = {'alpha': [i/10 for i in range(1, 1000, 1)]}

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(trainX, trainY)

# # Get the best hyperparameter
# best_lambda = grid_search.best_params_['alpha']

# # Fit Ridge regression model with the best hyperparameter on the training set
# best_ridge_model = Ridge(alpha=best_lambda)
# best_ridge_model.fit(trainX, trainY)

# # Predict on the scaled validation set
# y_pred_val = best_ridge_model.predict(valX)

# # Evaluate the model on the validation set using Mean Squared Error (MSE)
# mse_val = mean_squared_error(valY, y_pred_val)

# # Evaluate the model on the validation set using R-squared
# r2_val = r2_score(valY, y_pred_val)

# # Display the metrics on the validation set
# print(f"\nBest Lambda = {best_lambda}")
# print("Mean Squared Error on the Validation Set:", mse_val)
# print("R-squared on the Validation Set:", r2_val)



Lambda = 0.1
Mean Squared Error on the Validation Set: 12781864921.94652
R-squared on the Validation Set: 0.8752852588263048
Mean Absolute Error on the Validation Set: 80277.00501897825

Lambda = 0.3
Mean Squared Error on the Validation Set: 12728649530.282986
R-squared on the Validation Set: 0.8758044900838954
Mean Absolute Error on the Validation Set: 79992.02897610066

Lambda = 0.5
Mean Squared Error on the Validation Set: 12679752193.41179
R-squared on the Validation Set: 0.8762815893764644
Mean Absolute Error on the Validation Set: 79714.41053559569

Lambda = 0.7
Mean Squared Error on the Validation Set: 12634841373.500715
R-squared on the Validation Set: 0.8767197915727254
Mean Absolute Error on the Validation Set: 79443.77288920336

Lambda = 0.9
Mean Squared Error on the Validation Set: 12593619363.385025
R-squared on the Validation Set: 0.8771220014500501
Mean Absolute Error on the Validation Set: 79241.52440312982

Lambda = 1.1
Mean Squared Error on the Validation Set: 125558

 <span style="font-size:13pt;">Polynomial LBF with degree 2</span>

In [471]:
from sklearn.preprocessing import PolynomialFeatures

# Loop over different values of lambda
for lambda_val in lambda_values:
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=2)
    trainX_poly = poly_features.fit_transform(trainX)
    valX_poly = poly_features.transform(valX)

    # Scale and standardize the polynomial features
    scaler = StandardScaler()
    trainX_poly_scaled = scaler.fit_transform(trainX_poly)
    valX_poly_scaled = scaler.transform(valX_poly)

    # Fit a ridge regression model on the scaled polynomial training set
    ridge_model = Ridge(alpha=lambda_val)
    ridge_model.fit(trainX_poly_scaled, trainY)

    # Predict house prices on the scaled polynomial validation set
    y_pred_val = ridge_model.predict(valX_poly_scaled)

    # Evaluate the model on the validation set using Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared
    mse_val = mean_squared_error(valY, y_pred_val)
    mae_val = mean_absolute_error(valY, y_pred_val)
    r2_val = r2_score(valY, y_pred_val)

    # Display the metrics on the validation set for the current lambda value
    print(f"\nLambda = {lambda_val}")
    print("Mean Squared Error on the Validation Set:", mse_val)
    print("Mean Absolute Error on the Validation Set:", mae_val)
    print("R-squared on the Validation Set:", r2_val)


Lambda = 0.1
Mean Squared Error on the Validation Set: 154865189369.10608
Mean Absolute Error on the Validation Set: 233465.984615836
R-squared on the Validation Set: -0.5110449161312272

Lambda = 0.3
Mean Squared Error on the Validation Set: 61047410642.2053
Mean Absolute Error on the Validation Set: 157601.02589937882
R-squared on the Validation Set: 0.40435045558223015

Lambda = 0.5
Mean Squared Error on the Validation Set: 44291579647.45504
Mean Absolute Error on the Validation Set: 140612.8584722729
R-squared on the Validation Set: 0.567839832009017

Lambda = 0.7
Mean Squared Error on the Validation Set: 37632256815.43124
Mean Absolute Error on the Validation Set: 132348.44116559945
R-squared on the Validation Set: 0.6328159312292434

Lambda = 0.9
Mean Squared Error on the Validation Set: 33990180250.508274
Mean Absolute Error on the Validation Set: 127245.47858287102
R-squared on the Validation Set: 0.6683522664121673

Lambda = 1.1
Mean Squared Error on the Validation Set: 31628

 <span style="font-size:13pt;">Best MAE for each approach (Scaling Issue BTW) </span>

For Regular OLS regression :
Mean Absolute Error on the Validation Set: 80795.62894775019

For Ridge Regression at lambda = 10.5:
Mean Absolute Error on the Validation Set: 78032.71643759069

For Polynomial degree 2 LBF at lambda = 14.9:
Mean Absolute Error on the Validation Set: 91683.32322667677

For Polynomial degree 3 LBF at lambda = 14.9:
Mean Absolute Error on the Validation Set: 74018.08740916016

the polynomial degree 3 works best because it output the lowest MAE so it means that it performs best when it get it tested on new data and therefore generalizes better. Even though there is something wrong with my scaling.